In [1]:
#Setup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, pickle, joblib, os, uproot
from tqdm import tqdm_notebook

import rawio.IO as io
from scdmsPyTools.Traces.Filtering import *
from scdmsPyTools.Traces.Noise import *
from scdmsPyTools.TES.Templates import *
from scdmsPyTools.Traces.Stats import *
#from scdmsPyTools.BatTools.IO import *
rawdir = "/data/midas/"
procdir = "/data/midas/proc"

In [2]:
#Select a file.
series='48883'
csv = [x[0] for x in pd.read_csv('~/git/R76Firmware/DCRC_Compatibility/manual_48873.txt',dtype=str,comment='#').values.tolist()]
#txt = [x[0] for x in pd.read_csv('~/git/R76Firmware/DCRC_Compatibility/towertest.txt',dtype=str).values.tolist()]

#More setup, dependent on series number.
procseries=series
filepath = rawdir+series

events = io.getRawEvents(rawdir,series)
try:
    stamps = [f'{QI[0]:04X}' for QI in events['Z6']['QI'][events['Z6']['PA'].keys()[0][0]]]
except KeyError:
    print(events.keys(),len(events))

del events

The file is: /data/midas//48883/RUN48883_F0000.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0000.mid.gz
RawDataReader: Extracted series from ODB: 72406211317
The file is: /data/midas//48883/RUN48883_F0001.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0001.mid.gz
RawDataReader: Extracted series from ODB: 72406211317


RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.
RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.
RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.
RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.


The file is: /data/midas//48883/RUN48883_F0002.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0002.mid.gz
RawDataReader: Extracted series from ODB: 72406211317
The file is: /data/midas//48883/RUN48883_F0003.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0003.mid.gz
RawDataReader: Extracted series from ODB: 72406211317
The file is: /data/midas//48883/RUN48883_F0004.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0004.mid.gz
RawDataReader: Extracted ser

RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.
RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.


The file is: /data/midas//48883/RUN48883_F0005.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0005.mid.gz
RawDataReader: Extracted series from ODB: 72406211317
The file is: /data/midas//48883/RUN48883_F0006.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0006.mid.gz
RawDataReader: Extracted series from ODB: 72406211317
The file is: /data/midas//48883/RUN48883_F0007.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48883/RUN48883_F0007.mid.gz
RawDataReader: Extracted ser

RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.
RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.


print(csv['timestamp'],[f'{i:x}' for i in stamps])
print(type(csv['timestamp'][0]),type(f'{100:x}'))
print('1337' in csv['timestamp'].values)

In [3]:
print(stamps,"\n====\n",csv,len(stamps),len(csv))

['8001', '0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '000A', '000B', '000C', '000D', '000E', '000F', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '001A', '001B', '001C', '001D', '001E', '001F', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '002A', '002B', '002C', '002D', '0030', '0031', '0032', '0033', '0034', '0035', '0036', '0037', '003A', '003D', '003E', '003F', '0040', '0041', '0042', '0043', '0046', '0049', '004C', '004F', '0052', '0054', '0055', '0056', '0057', '0058', '0059', '005A', '005B', '005C', '005D', '005E', '005F', '0061', '0062', '0063', '0065', '0066', '0067', '0068', '0069', '006A', '006B', '006C', '006D', '006E', '006F', '0070', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0078', '0079', '007A', '007B', '007C', '007D', '007E', '007F', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0088', '0089', '008A', '008B', '008C', '008D', '008E',

In [8]:
print(hex(len(stamps)),len(stamps)/385)

0x141 0.8337662337662337


In [5]:
shared = [x for x in csv if x in stamps]
print(shared)
csvshared = csv[csv.index(shared[0]):]#csv.index(shared[-1])+1]
stampshared = stamps[stamps.index(shared[0]):]#stamps.index(shared[-1])+1]
#stampshared = [x for x in stampshared if x != '014A']

[]


IndexError: list index out of range

In [ ]:
print(csvshared,stampshared)

In [ ]:
for x in list(zip(csvshared,stampshared)): 
    if(x[0][:2] == x[1][:2]): print(x,"*")
    else: print(x)
print(shared)

In [ ]:
print(len(stamps)-len(set(stamps)))
print(stamps)

In [6]:
repeats = [x for x in stamps if stamps.count(x) > 1]
print(set(repeats), (len(repeats)-len(set(repeats)))/len(set(repeats)))

for x in stamps:
    print(x+(stamps.count(x)-1)*"*")

ZeroDivisionError: division by zero

So it seems like none of these are aligned. I'm switching to using a FIFO instead of RAM so that I can store a handful of values -- maybe it's getting overwritten by one of the next few before it has a chance to write to the packet, so a different value is ultimately saved.
- On next iteration, just set the FIFO to max size -- we actually need more space than we have now, since the rate could go much higher, and if it's too small then the whole system will fail. There's a limit to this, where if we can handle that trigger rate, we can handle anything above it (because of the inhibit function), but there's not really any reason to try to keep this small, is there?

In [ ]:
for i,x in enumerate(stamps[stamps.index(shared[0]):]):
    print("{0:016b}".format(int(x,16) ^ int(csv[csv.index(shared[0]):][i],16)))

In [ ]:
for i,x in enumerate(stamps[stamps.index(shared[0]):]):
    print(x == csv[csv.index(shared[0]):][i])

Upside: fewer obvious red flag values like beef or 1337. I think the repetition is a red flag. Is there anything in the code that tells the DCRC to retrieve the same value again, maybe?

In [ ]:
print(kill_dupes)

# Algorithm
from R76Analysis/example_scripts/time_difference...

In [ ]:
from itertools import zip_longest

bold_text = "\033[1m"
reset_format = "\033[0m"
title = "Merge and look for doubles:"
print(f"{bold_text}{title}{reset_format}")
print("pros and cons: this method is very simple and sorts well, but it deletes a lot of good data too")

merged_list = [item for pair in zip_longest(stamps,csv, fillvalue=None) for item in pair if item is not None]

for i,s in enumerate(merged_list):
    print(i,s)

trash=set()
i = 0
while i + 2 < len(merged_list):
    if merged_list[i] == merged_list[i + 1]:
        #print(":)")
        i += 2
    else:
        trash.add(i)
        trash.add(i + 1)
        trash.add(i + 2)
        i += 3
            
for i in sorted(trash, reverse=True):
    del merged_list[i]

print()
print("New:")
for i,s in enumerate(merged_list):
    print(i,s)

todo:
- let it search the rest of the list, in case there are extras to be dropped
- tolerance +/- 1?